### Exploratory Data Analysis (EDA)

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import tempfile

import ads
from ads import set_auth
from ads.dataset.factory import DatasetFactory

In [4]:
# data
TARGET = 'Attrition'

attrition_path = "/opt/notebooks/ads-examples/oracle_data/orcl_attrition.csv"

### Carico i dati utilizzando un ADS Dataset

In [5]:
ds = DatasetFactory.open(attrition_path, target=TARGET).set_positive_class('Yes')

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

### ma se voglio posso in ogni momento lavorare su Padas DataFrame

In [6]:
type(ds.to_pandas())

pandas.core.frame.DataFrame

In [7]:
ds.show_in_notebook()

Accordion(children=(HTML(value='<h3>Type: BinaryClassificationDataset</h3><h3>1,470 Rows, 36 Columns</h3><h4>C…

In [8]:
ds.head()

,Age,Attrition,TravelForWork,SalaryLevel,JobFunction,CommuteLength,EducationalLevel,EducationField,Directs,EmployeeNumber,...,WeeklyWorkedHours,StockOptionLevel,YearsinIndustry,TrainingTimesLastYear,WorkLifeBalance,YearsOnJob,YearsAtCurrentLevel,YearsSinceLastPromotion,YearsWithCurrManager,name
0,42,True,infrequent,5054,Product Management,2,L2,Life Sciences,1,1,...,80,0,8,0,1,6,4,0,5,Tracy Moore
1,50,False,often,1278,Software Developer,9,L1,Life Sciences,1,2,...,80,1,10,3,3,10,7,1,7,Andrew Hoover
2,38,True,infrequent,6296,Software Developer,3,L2,Other,1,4,...,80,0,7,3,3,0,0,0,0,Julie Bell
3,34,False,often,6384,Software Developer,4,L4,Life Sciences,1,5,...,80,0,8,3,3,8,7,3,0,Thomas Adams
4,28,False,infrequent,2710,Software Developer,3,L1,Medical,1,7,...,80,1,6,3,3,2,2,2,2,Johnathan Burnett


In [9]:
ds.numeric_columns

array(['Age', 'SalaryLevel', 'CommuteLength', 'Directs', 'EmployeeNumber',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate',
       'NumCompaniesWorked', 'PercentSalaryHike',
       'RelationshipSatisfaction', 'WeeklyWorkedHours',
       'StockOptionLevel', 'YearsinIndustry', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsOnJob', 'YearsAtCurrentLevel',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'], dtype=object)

### Categoriche e numeriche: importante per LGBM e per le trasformazioni

In [10]:
#
# definisco le funzioni che identificano le categorie di colonne
#
def cat_cols_selector(df, target_name):
    # the input is the dataframe
    
    # cols with less than THR values are considered categoricals
    THR = 10
    
    nunique = df.nunique()
    types = df.dtypes
    
    col_list = []
    
    for col in df.columns:
        if ((types[col] == 'object') or (nunique[col] < THR)):
            # print(col)
            if col != target_name:
                col_list.append(col)
    
    return col_list

def num_cols_selector(df, target_name):
    THR = 10
    
    types = df.dtypes
    nunique = df.nunique()
    
    col_list = []
    
    for col in df.columns:
        if (types[col] != 'object') and (nunique[col] >= THR): 
            # print(col)
            if col != target_name:
                col_list.append(col)
    
    return col_list

In [18]:
cat_cols = list(cat_cols_selector(ds.to_pandas(), TARGET))
num_cols = list(num_cols_selector(ds.to_pandas(), TARGET))

print(f"List of categorical columns: {cat_cols}, [{len(cat_cols)}]")
print()
print(f"List of nymerical columns: {num_cols}, [{len(num_cols)}]")

List of categorical columns: ['TravelForWork', 'JobFunction', 'EducationalLevel', 'EducationField', 'Directs', 'EnvironmentSatisfaction', 'Gender', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'Over18', 'OverTime', 'PerformanceRating', 'RelationshipSatisfaction', 'WeeklyWorkedHours', 'StockOptionLevel', 'TrainingTimesLastYear', 'WorkLifeBalance', 'name'], [21]

List of nymerical columns: ['Age', 'SalaryLevel', 'CommuteLength', 'EmployeeNumber', 'HourlyRate', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'YearsinIndustry', 'YearsOnJob', 'YearsAtCurrentLevel', 'YearsSinceLastPromotion', 'YearsWithCurrManager'], [14]
